In [70]:
from __future__ import print_function

from sklearn.preprocessing import OneHotEncoder
from keras.layers.core import Dense, Activation, Dropout
from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import Dense, Embedding, Bidirectional, Input, TimeDistributed
from keras.layers import LSTM
from keras.datasets import imdb
from keras import optimizers
import pandas as pd
import numpy as np
import os

In [71]:
#parameters
maxlen = 30
labels = 2

In [96]:
czech = pd.read_excel('czech.xlsx', encoding='latin',header = None)
czech.columns = ['Name', 'm_or_f']
czech['namelen'] = [len(str(i)) for i in czech['Name']]


In [97]:
cname = czech['Name']
collect = []


for i in range(len(cname)):
    collect.extend(list(str(cname[i]).lower()))
# collect.extend(['END'])
collect = set(collect)

czech.head()

,Name,m_or_f,namelen
0,Abigail,f,7
1,Ada,f,3
2,Adalberta,f,9
3,Adéla,f,5
4,Adelaida,f,8


In [98]:
data_set = pd.read_csv("gender_data.csv",header=None)
data_set.columns = ['name','m_or_f']
data_set['namelen']= [len(str(i)) for i in data_set['name']]
data_set1 = data_set[(data_set['namelen'] >= 2) ]

In [99]:
data_set1.groupby('m_or_f')['name'].count()

m_or_f
f    6705
m    8475
Name: name, dtype: int64

In [100]:
names = data_set['name']
gender = data_set['m_or_f']
vocab = set(' '.join([str(i) for i in names]))
vocab.add('END')
vocab = vocab.union(collect)
len_vocab = len(vocab)

In [101]:
print(vocab)
print("vocab length is ",len_vocab)
print ("length of data_set is ",len(data_set1))

{'r', '5', 'i', '4', '8', 'x', '3', 'č', 'ž', 'ň', 's', 'b', 'š', 'ú', '7', '2', 'l', 'j', 'g', 'h', 'ó', 'ý', '0', 'END', 'ď', 'w', 'c', 'a', 'k', '6', 'm', 'f', 'u', 'á', 'ě', 'é', 'ů', '1', 'ř', 't', 'q', ' ', 'p', 'y', 'í', 'n', 'v', 'e', '.', 'o', 'd', 'z', '9', 'ť'}
vocab length is  54
length of data_set is  15226


In [102]:
char_index = dict((c, i) for i, c in enumerate(vocab))

In [103]:
print(char_index)

{'r': 0, 'í': 44, '5': 1, 'i': 2, '4': 3, '8': 4, 'x': 5, '3': 6, 'ý': 21, 'č': 7, 'ž': 8, 'ň': 9, 's': 10, 'b': 11, 'š': 12, 'ú': 13, '7': 14, '2': 15, 'ď': 24, 'l': 16, 'j': 17, 'd': 50, 'g': 18, 'h': 19, 'ó': 20, '0': 22, 'e': 47, 'ů': 36, 'w': 25, 'c': 26, 'a': 27, 'k': 28, 'm': 30, 'f': 31, 'u': 32, 'á': 33, 'ě': 34, 'é': 35, '1': 37, 'ř': 38, 't': 39, '6': 29, ' ': 41, 'p': 42, 'q': 40, 'n': 45, 'v': 46, '.': 48, 'o': 49, 'y': 43, 'z': 51, 'END': 23, '9': 52, 'ť': 53}


In [104]:
#train test split
msk = np.random.rand(len(data_set1)) < 0.8
train = data_set1[msk]
test = data_set1[~msk]     


In [105]:
def set_flag(i):
    tmp = np.zeros(len_vocab);
    tmp[i] = 1
    return(tmp)

In [106]:
set_flag(3)

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

#### modify the code above to also convert each index to one-hot encoded representation

In [113]:
#take data_set upto max and truncate rest
#encode to vector space(one hot encoding)
#padd 'END' to shorter sequences
#also convert each index to one-hot encoding
train_X = []
train_Y = []
trunc_train_name = [str(i)[0:maxlen] for i in train.name]
for i in trunc_train_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    train_X.append(tmp)
for i in train.m_or_f:
    if i == 'm':
        train_Y.append([1,0])
    else:
        train_Y.append([0,1])
    
train_X=np.asarray(train_X)
train_Y=np.asarray(train_Y)

In [110]:
test_X = []
test_Y = []
trunc_test_name = [str(i)[0:maxlen] for i in test.name]
for i in trunc_test_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    test_X.append(tmp)
for i in test.m_or_f:
    if i == 'm':
        test_Y.append([1,0])
    else:
        test_Y.append([0,1])
    
test_X = np.asarray(test_X)
test_Y = np.asarray(test_Y)

In [111]:
print(np.asarray(test_X).shape)
print(np.asarray(test_Y).shape)

(3136, 30, 54)
(3136, 2)


In [116]:
msk = np.random.rand(len(czech)) < 0.8


vtrain = czech[msk]
vtest = czech[~msk]

In [122]:
vtrain_x = []
vtrain_y = []

train_name = [str(i) for i in vtrain.Name]
for i in train_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0, maxlen - len(str(i))):
        tmp.append(set_flag(char_index['END']))
    vtrain_x.append(tmp)
for i in vtrain.m_or_f:
    if i == 'm':
        vtrain_y.append([1,0])
    else:
        vtrain_y.append([0,1])
vtrain_x = np.asarray(vtrain_x)
vtrain_y = np.asarray(vtrain_y)

In [123]:
vtest_x = []
vtest_y = []

train_name = [str(i) for i in vtest.Name]
for i in train_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0, maxlen - len(str(i))):
        tmp.append(set_flag(char_index['END']))
    vtest_x.append(tmp)
for i in vtest.m_or_f:
    if i == 'm':
        vtest_y.append([1,0])
    else:
        vtest_y.append([0,1])
vtest_x = np.asarray(vtest_x)
vtest_y = np.asarray(vtest_y)

#### build model in keras ( a stacked LSTM model with many-to-one arch ) here 30 sequence and 2 output each for one category(m/f)

In [120]:
#build the model: 2 stacked LSTM
print('Build model...')
input_bilstm=Input(shape = (maxlen,len_vocab))
bi_one = Bidirectional(LSTM(512, return_sequences=True))(input_bilstm)
drop1 = Dropout(0.2)(bi_one)
bi_two = Bidirectional(LSTM(512, return_sequences=False))(drop1)
drop2 = Dropout(0.2)(bi_two)
output = Dense(2, activation='softmax')(drop2)
model = Model(input_bilstm, output)


optimizer = optimizers.adam(lr = 0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])




Build model...


In [121]:
batch_size=500
model.fit(train_X, train_Y,
          batch_size=batch_size,
          epochs=10,
          validation_data=(vtrain_x, vtest_y)
         )
model.save('Martin_program.h5')

Train on 12090 samples, validate on 995 samples
Epoch 1/10
12090/12090 [==============================] - 15s 1ms/step - loss: 0.6216 - acc: 0.6471 - val_loss: 0.6406 - val_acc: 0.6452
Epoch 2/10
12090/12090 [==============================] - 12s 1ms/step - loss: 0.5135 - acc: 0.7495 - val_loss: 0.5071 - val_acc: 0.7638
Epoch 3/10
12090/12090 [==============================] - 12s 1ms/step - loss: 0.4971 - acc: 0.7589 - val_loss: 0.5859 - val_acc: 0.6925
Epoch 4/10
12090/12090 [==============================] - 13s 1ms/step - loss: 0.4928 - acc: 0.7676 - val_loss: 0.4842 - val_acc: 0.7678
Epoch 5/10
12090/12090 [==============================] - 13s 1ms/step - loss: 0.4636 - acc: 0.7812 - val_loss: 0.4872 - val_acc: 0.7779
Epoch 6/10
12090/12090 [==============================] - 13s 1ms/step - loss: 0.4322 - acc: 0.8050 - val_loss: 0.3835 - val_acc: 0.8382
Epoch 7/10
12090/12090 [==============================] - 13s 1ms/step - loss: 0.4217 - acc: 0.8127 - val_loss: 0.4061 - val_acc: 

In [124]:
score, acc = model.evaluate(vtest_x, vtest_y)
print('Test score:', score)
print('Test accuracy:', acc)

255/255 [==============================] - 0s 936us/step
Test score: 0.33718277274393565
Test accuracy: 0.8588235296455085


In [148]:
name=["riya"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i.lower())]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[0.16779284, 0.83220714]], dtype=float32)

In [47]:
pred

array([[0.05657073, 0.9434293 ],
       [0.9473468 , 0.05265319],
       [0.9572235 , 0.04277651]], dtype=float32)

#### Lets train more, clearly some very simple female names it doesnt get right like mentioned above (inspite it exists in training data)

In [48]:
score, acc = model.evaluate(test_X, test_Y)
print('Test score:', score)
print('Test accuracy:', acc)

3108/3108 [==============================] - 2s 771us/step
Test score: 0.43282059709193305
Test accuracy: 0.8841698841698842


In [49]:
name=["sandhya","jaspreet","rajesh","kaveri","aditi deepak","arihant","sasikala","aditi","ragini rajaram"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[5.6570753e-02, 9.4342929e-01],
       [9.4734681e-01, 5.2653167e-02],
       [9.5722347e-01, 4.2776532e-02],
       [3.5088859e-02, 9.6491110e-01],
       [2.1997560e-04, 9.9978000e-01],
       [9.9858874e-01, 1.4111887e-03],
       [1.1531739e-04, 9.9988472e-01],
       [4.6136803e-03, 9.9538630e-01],
       [1.8581923e-02, 9.8141807e-01]], dtype=float32)

In [33]:
name=["abhi","abhi deepak","mr. abhi"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[0.4178515 , 0.5821485 ],
       [0.37405205, 0.62594795],
       [0.02188767, 0.9781123 ]], dtype=float32)

In [34]:
name=["rajini","rajinikanth","mr. rajini"]
X=[]
trunc_name = [i[0:maxlen] for i in name]
for i in trunc_name:
    tmp = [set_flag(char_index[j]) for j in str(i)]
    for k in range(0,maxlen - len(str(i))):
        tmp.append(set_flag(char_index["END"]))
    X.append(tmp)
pred=model.predict(np.asarray(X))
pred

array([[0.09026915, 0.90973085],
       [0.9785337 , 0.02146639],
       [0.03325909, 0.9667409 ]], dtype=float32)

In [35]:
#save our model and data
model.save_weights('gender_model',overwrite=True)
train.to_csv("train_split.csv")
test.to_csv("test_split.csv")

In [36]:
evals = model.predict(test_X)
prob_m = [i[0] for i in evals]

In [37]:
out = pd.DataFrame(prob_m)
out['name'] = test.name.reset_index()['name']
out['m_or_f']=test.m_or_f.reset_index()['m_or_f']

In [38]:
out.head(10)
out.columns = ['prob_m','name','actual']
out.head(10)
out.to_csv("gender_pred_out.csv")

In [150]:
check = pd.read_csv('name_gender.csv')

In [151]:
check.shape

(95026, 3)

In [152]:
check.head()


,name,gender,probability
0,Aaban,M,1.0
1,Aabha,F,1.0
2,Aabid,M,1.0
3,Aabriella,F,1.0
4,Aada,F,1.0


In [ ]:
check = check[]